In [1]:
#!/usr/bin/python
# -*- coding: sjis -*-
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pickle
import cv2
import random
from multiboxloss import MultiBoxLoss
from mydataloader2 import *
from augmentations import SSDAugmentation

from tqdm.notebook import tqdm

In [2]:
#http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar

In [3]:
#https://s3.amazonaws.com/amdegroot-models/vgg16_reducedfc.pth

In [4]:
## (1) データの準備と設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 64 #30
augment = SSDAugmentation()
prepro = PreProcess(augment)
dirpath = './VOCdevkit/VOC2012/JPEGImages/'
epoch_num = 15

In [5]:
## (2) モデルの定義
from mynet import SSD

In [6]:
## (3) モデルの生成，損失関数，最適化関数の設定
##   (3.1) モデルの生成
net = SSD()
vgg_weights = torch.load('./models/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)
net.to(device)

##   (3.2) 損失関数 の設定
optimizer = optim.SGD(net.parameters(),
                      lr=1e-3,momentum=0.9,
                      weight_decay=5e-4)

##   (3.3) 最適化関数の設定
from multiboxloss import MultiBoxLoss
criterion = MultiBoxLoss(device=device)

In [9]:
## (4) 学習
net.train()
for ep in tqdm(range(epoch_num)):
    i = 0
    ans = pickle.load(open('ans.pkl', 'rb'))
    dataset = MyDataset(ans, dirpath, prepro)
    dataloader = DataLoader(dataset,batch_size=batch_size, 
                            shuffle=True, collate_fn=my_collate_fn)
    for xs, ys in tqdm(dataloader):
        xs  = [ torch.FloatTensor(x) for x in xs ]        
        images = torch.stack(xs, dim=0)
        images = images.to(device)        
        targets  = [ torch.FloatTensor(y).to(device) for y in ys ]        
        outputs = net(images)
        loss_l, loss_c = criterion(outputs, targets)
        loss = loss_l + loss_c
        print(i, loss_l.item(), loss_c.item())
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_value_(net.parameters(), clip_value=2.0)
        optimizer.step()
        loss_l, loss_c  = 0, 0
        xs, ys, bc = [], [], 0
        i += 1        
    outfile = "./models/ssd3-" + str(ep) + ".model"
    torch.save(net.state_dict(),outfile)
    print(outfile," saved")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/191 [00:00<?, ?it/s]

/work/F/pytorch-object-detection-program/program/test1/augmentations.py:240: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


0 2.73567271232605 16.4898624420166
1 2.5330569744110107 15.613561630249023
2 2.582627773284912 14.135025978088379
3 2.6395750045776367 12.79157829284668
4 2.6575093269348145 12.414515495300293
5 2.7552263736724854 12.34683609008789
6 2.5352492332458496 12.368987083435059


Corrupt JPEG data: premature end of data segment


...

0 2.0385208129882812 4.444845676422119
1 1.8681102991104126 4.270753860473633
2 2.0646591186523438 4.760782241821289
3 1.8771271705627441 5.1347150802612305
4 1.8392400741577148 4.9399733543396
5 2.0073654651641846 4.4162397384643555
6 1.9308134317398071 4.915396690368652
7 2.1122963428497314 4.705398082733154
8 2.1435883045196533 5.210374355316162
9 2.2626230716705322 4.509541988372803
10 1.9410074949264526 4.765841960906982
11 1.8261332511901855 4.791759014129639
12 1.9383610486984253 4.723002910614014
13 2.1305341720581055 4.951857566833496
14 2.0041632652282715 4.459288120269775
15 1.9945100545883179 4.914962291717529
16 1.9106462001800537 4.553580284118652
17 1.9478347301483154 4.673267364501953
18 1.8536051511764526 4.829941272735596
19 1.9823384284973145 4.621645450592041
20 2.094468832015991 4.31493616104126
21 2.062195301055908 5.021909236907959
22 2.0744853019714355 4.230087757110596
23 2.13584303855896 4.937096118927002
24 2.1355974674224854 4.748419761657715
25 1.9609742164

Corrupt JPEG data: 349 extraneous bytes before marker 0xd9


41 1.7679811716079712 4.550726413726807
42 2.021979331970215 4.9391703605651855
43 1.8728289604187012 4.6363067626953125
44 1.9947506189346313 4.584421634674072
45 1.8723564147949219 4.739198684692383
46 1.8233169317245483 4.900681018829346
47 1.7807750701904297 4.443795204162598
48 1.9792689085006714 4.669641494750977
49 1.8393656015396118 4.658182144165039
50 2.0164365768432617 4.256960868835449
51 1.8609189987182617 4.3706793785095215
52 1.811273455619812 4.616129398345947
53 1.9787042140960693 4.745967864990234
54 2.0241453647613525 4.489486217498779
55 1.9709218740463257 4.834067344665527
56 2.1065237522125244 4.690495491027832
57 2.0653598308563232 4.664525508880615
58 2.047572374343872 4.547375202178955
59 2.1207823753356934 4.511150360107422
60 1.883245825767517 4.40794563293457
61 1.8993022441864014 4.6883015632629395
62 2.027345657348633 4.241824626922607
63 1.9947344064712524 5.136812210083008
64 2.0690033435821533 4.664374351501465
65 1.8896162509918213 4.561890125274658
66

Corrupt JPEG data: premature end of data segment


181 1.7108361721038818 4.6017746925354
182 1.8006911277770996 4.373085021972656
183 1.9477931261062622 4.6281280517578125
184 1.7619270086288452 4.559062957763672
185 1.807905912399292 4.428924560546875
186 1.9339426755905151 4.088348865509033
187 1.6565483808517456 4.476376533508301
188 1.3636759519577026 4.288267612457275
189 1.7140358686447144 4.537351131439209
190 2.0073084831237793 4.5449299812316895
ssd3-2.model  saved
